In [1]:
import time
import os
import paramiko
from io import StringIO
import time
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from IPython.display import clear_output
import datetime

In [40]:
# SSH credentials and remote CSV file path
# host = 'lxslc7.ihep.ac.cn'
# username = os.environ.get('IHEP_USERNAME')
# password = os.environ.get('IHEP_PASSWORD')
# remote_csv_file = '/hpcfs/cepc/higgs/frank/GNN/ds/output_tag/summaries_0.csv'
# remote_csv_file = '/hpcfs/cepc/higgs/frank/GNN/ds/output_rec/summaries_0.csv'


host = 'bl-0.inpac.sjtu.edu.cn'
username = os.environ.get('SJTU_USERNAME')
password = os.environ.get('SJTU_PASSWORD')
remote_csv_file = '/lustre/collider/zhangyulei/DeepLearning/Tracking/workspace/output/summaries_0.csv'

# Establish an SSH connection
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(host, username=username, password=password)
print(f'Successfully connected to {username}@{host}')

Successfully connected to zhangyulei@bl-0.inpac.sjtu.edu.cn


In [41]:
def update_csv(remote_file: str = None):
    sftp = ssh.open_sftp()

    if remote_file is None:
        remote_file = remote_csv_file

    with sftp.open(remote_file, 'r') as file:
        remote_csv_data = file.read().decode('utf-8')
        timestamp = sftp.stat(remote_file).st_mtime

        dt_object = datetime.datetime.fromtimestamp(timestamp)

        return pd.read_csv(StringIO(remote_csv_data)), \
            f"{dt_object.year} - {dt_object.month} - {dt_object.day}   {dt_object.hour} : {dt_object.minute}"


def update_fig(df, t):
    fig = go.Figure()

    y_max, y_min = df.max()[['train_loss', 'valid_loss']].max(), df.min()[['train_loss', 'valid_loss']].min()

    df_new = df[['epoch', 'train_loss', 'valid_loss', 'valid_dp_mean', 'valid_dp_std', 'valid_acc']]. \
        groupby('epoch').transform("mean").drop_duplicates(keep='last', subset=['train_loss'])

    mu_max = df.abs().max()['valid_dp_mean']
    std_max = df.abs().max()['valid_dp_std']

    fig.data = []
    fig.add_trace(
        go.Scatter(
            x=df_new.index,
            y=df_new['valid_dp_mean'],
            mode='lines', name="itr: p mean",
            line=dict(dash='dot', color="#ff7f0e"), yaxis='y3',
            legendgroup="Momentum",
            legendgrouptitle_text="Momentum",
        )
    )
    fig.add_trace(
        go.Scatter(
            x=df_new.index,
            y=df_new['valid_dp_std'],
            mode='lines', name="itr: p std",
            line=dict(dash='dot', color="#64a05f"), yaxis='y4',
            legendgroup="Momentum",
        )
    )

    fig.add_trace(
        go.Scatter(
            x=df.index, y=df['train_loss'], mode='lines', name="itr: train", legendgroup="Itr Loss",
            legendgrouptitle_text="Itr Loss", line=dict(dash='dot'))
    )
    fig.add_trace(
        go.Scatter(
            x=df.index, y=df['valid_loss'], mode='lines', name="itr: valid", legendgroup="Itr Loss",
            line=dict(dash='dot')
        ))
    fig.add_trace(
        go.Scatter(
            x=df_new.index,
            y=-np.log10(1 - df_new['valid_acc']),
            # y=df['valid_acc'],
            mode='lines', name="itr: accuracy",
            line=dict(dash='dot', color="#11ADF0"), yaxis='y2',
            legendgroup="Edge Label",
            legendgrouptitle_text="Edge Label",
        )
    )

    fig.add_trace(go.Scatter(
        x=df_new.index, y=df_new['train_loss'], mode='lines+markers', name="epoch: train",
        legendgroup="Epoch Loss", legendgrouptitle_text="Epoch Loss"
    ))
    fig.add_trace(go.Scatter(
        x=df_new.index, y=df_new['valid_loss'], mode='lines+markers', name="epoch: valid",
        legendgroup="Epoch Loss"
    ))

    for lr in df.drop_duplicates(keep="first", subset=['lr'])['lr'].items():
        print(lr)
        fig.add_vline(x=lr[0], line_width=2, line_dash="dash", line_color="grey")
        fig.add_annotation(
            text=f'$\eta = {lr[1]}$',
            x=lr[0] + 0.5, y=np.log10(y_min) * 1.02,  # Set the position using numeric values
            xanchor='left',
            # yanchor='bottom',
            showarrow=False,
            font=dict(size=14, color='grey')
        )

    fig.update_layout(
        title=f"{t} --> Epoch: {df['epoch'].iloc[-1]}, Iteration: {df['itr'].iloc[-1]}",
        width=1200,
        height=700,
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        xaxis=dict(
            title_text="Total iteration",
            showgrid=False,
            mirror=True,
            showline=True,
            zeroline=False,
            linewidth=2,
            linecolor='#666666', gridcolor='#d9d9d9',
            domain=[0.1, 0.85],
        ),
        yaxis=dict(
            title=r"Loss",
            titlefont=dict(color="#d62728"),
            tickfont=dict(color="#d62728"),
            showgrid=False,
            linecolor="#d62728", gridcolor='#d9d9d9',
            zeroline=False,
            type="log",
            range=[np.log10(y_min) * 1.05, np.log10(y_max) * 1.05],
        ),
        yaxis2=dict(
            # title=r"$\textrm{95% CI }{\Large \kappa_{2V}}\textrm{ Interval}$",
            title=r"$-log(\epsilon_{error})$",
            titlefont=dict(color="#11ADF0"),
            tickfont=dict(color="#11ADF0"),
            anchor="free",
            overlaying="y",
            side="left",
            position=0.0,
            showgrid=False,
            linecolor="#11ADF0", gridcolor='#d9d9d9',
            zeroline=False,
            # type="log",
            # range=[3.0, 3.7],
        ),
        yaxis3=dict(
            title=r"$\mu^{p}_{\textrm{valid}}$",
            titlefont=dict(color="#ff7f0e"),
            tickfont=dict(color="#ff7f0e"),
            anchor="x",
            overlaying="y",
            side="right",
            showgrid=False,
            linecolor="#ff7f0e", gridcolor='#d9d9d9',
            zeroline=False,
            range=[-mu_max * 1.05, mu_max * 1.05]
        ),
        yaxis4=dict(
            title=r"$\sigma^{p}_{\textrm{valid}}$",
            titlefont=dict(color="#64a05f"),
            tickfont=dict(color="#64a05f"),
            anchor="free",
            overlaying="y",
            side="right",
            position=0.95,
            showgrid=False,
            linecolor="#64a05f", gridcolor='#d9d9d9',
            zeroline=False,
            # range=[0.0, std_max],
            range=[0.0, 1.0],
        ),
    )

    return fig

In [42]:
tag = '/hpcfs/cepc/higgs/frank/GNN/ds/output_tag/summaries_0.csv'
rec = '/hpcfs/cepc/higgs/frank/GNN/ds/output_rec/summaries_0.csv'

# tar = tag
tar = rec

# df, t = update_csv('/hpcfs/cepc/higgs/frank/GNN/ds/output_rec.gradnorm.4.newloss/summaries_0.csv')
# df, t = update_csv('/hpcfs/cepc/higgs/frank/GNN/ds/output_rec.gradnorm.4/summaries_0.csv')
df, t = update_csv('/lustre/collider/zhangyulei/DeepLearning/Tracking/workspace/output.rec.trans/summaries_0.csv')
fig = update_fig(df, t)
fig.show()
#
# while True:
#     df, t = update_csv(tar)
#     fig = update_fig(df, t)
#     time.sleep(120)
#     clear_output(wait=False)
#     fig.show()

(0, 0.005)
(18, 0.0070710678118654)
(360, 0.0007071067811865)
(630, 0.0001414213562373)


In [43]:
df_new = df[[
    'epoch',
    'valid_num_tracks_diff_0',
    'valid_num_tracks_diff_1',
    'valid_num_tracks_diff_2',
    'valid_num_tracks_diff_-1',
    'valid_num_tracks_diff_-2',
    'valid_num_tracks_diff_overflow',
    'valid_num_tracks_diff_underflow',
    'valid_TP',
    'valid_TN',
    'valid_FP',
    'valid_FN',
]].groupby('epoch', as_index=False).transform("sum").drop_duplicates(keep='last')

In [44]:
def update_stack(df: pd.DataFrame, columns, t):
    fig = go.Figure()

    norm_sum = df[columns].sum(axis=1)

    for key in columns:
        fig.add_trace(go.Bar(
            name=key,
            y=df[key]/norm_sum,
            x=df['epoch'],
            width=1,
            offset=0,
        ))

    fig.update_layout(
        title=f"{t} --> Epoch: {df['epoch'].iloc[-1]}",
        width=800,
        height=700,
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        barmode="stack",
        # yaxis=dict(
        #     type="log",
        # ),
        
    )

    fig.show()

update_stack(df_new, [col for col in df_new.columns if 'valid_num_tracks_diff' in col], t)

update_stack(df_new, [col for col in df_new.columns if 'valid_num_tracks_diff' not in col][1:], t)


In [33]:
df

,lr,train_loss,l1,l2,grad_norm,train_batches,itr,epoch,valid_loss,valid_acc,...,valid_sum_total,valid_dp_mean,valid_dp_std,valid_num_tracks_diff_0,valid_num_tracks_diff_1,valid_num_tracks_diff_2,valid_num_tracks_diff_-1,valid_num_tracks_diff_-2,valid_num_tracks_diff_underflow,valid_num_tracks_diff_overflow
0,0.005000,1.134064,32704.488811,60.379101,0.741356,36,0,0,0.466785,0.916028,...,699941.532715,-999.0,NaN,6,0,0,238,712,21,0
1,0.005000,0.389768,32716.914959,60.616954,0.974830,35,1,0,0.344978,0.940478,...,771868.105469,-999.0,NaN,72,19,0,327,495,27,0
2,0.005000,0.379525,32656.028619,60.582843,0.552960,6,2,0,0.310934,0.942483,...,150110.953125,-999.0,NaN,5,2,0,51,92,9,0
3,0.005000,0.312439,32304.930606,60.388347,3.212250,41,3,0,0.302837,0.946333,...,747961.822266,-999.0,NaN,68,8,0,378,643,19,0
4,0.005000,0.299074,32757.925487,61.176396,0.678878,36,4,0,0.276379,0.951794,...,737885.597656,-999.0,NaN,91,18,1,369,487,12,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692,0.000141,0.058151,10205.878273,54.838183,0.150261,49,8,38,0.064351,0.988492,...,672280.605469,-999.0,NaN,72,4,1,448,791,15,0
693,0.000141,0.049362,10205.420962,54.838061,0.448686,2,9,38,0.038909,0.993222,...,29695.832031,-999.0,NaN,8,0,0,22,19,0,0
694,0.000141,0.039482,10189.906634,54.832758,0.210339,35,10,38,0.052752,0.990020,...,763668.843750,-999.0,NaN,356,31,3,404,122,24,0
695,0.000141,0.051407,10185.742517,54.830576,0.163239,11,11,38,0.074435,0.984904,...,161656.898438,-999.0,NaN,14,0,0,93,189,3,0
